In [7]:
#/Users/rahmi/Desktop/Tuutkia/unlabeledTrainData.tsv

import pandas as pd

# Read data from files 
train = pd.read_csv( "/Users/rahmi/Desktop/Tuutkia/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "/Users/rahmi/Desktop/Tuutkia/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "/Users/rahmi/Desktop/Tuutkia/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [8]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,'lxml').get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [9]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [10]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [11]:
print (len(sentences))

795538


In [12]:
print (sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [14]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-03-03 18:53:13,738 : INFO : 'pattern' package not found; tag filters are not available for English
2018-03-03 18:53:13,773 : INFO : collecting all words and their counts
2018-03-03 18:53:13,774 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-03 18:53:13,972 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


Training model...


2018-03-03 18:53:14,120 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-03-03 18:53:14,349 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-03-03 18:53:14,590 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-03-03 18:53:14,757 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-03-03 18:53:14,911 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-03-03 18:53:15,086 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-03-03 18:53:15,248 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-03-03 18:53:15,394 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-03-03 18:53:15,527 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50

2018-03-03 18:53:25,533 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2018-03-03 18:53:25,690 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-03-03 18:53:25,862 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-03-03 18:53:25,999 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-03-03 18:53:26,128 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-03-03 18:53:26,288 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-03-03 18:53:26,396 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-03-03 18:53:26,398 : INFO : Loading a fresh vocabulary
2018-03-03 18:53:26,752 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-03-03 18:

2018-03-03 18:54:23,793 : INFO : EPOCH 2 - PROGRESS: at 77.94% examples, 374921 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:54:24,809 : INFO : EPOCH 2 - PROGRESS: at 80.08% examples, 370950 words/s, in_qsize 6, out_qsize 1
2018-03-03 18:54:25,814 : INFO : EPOCH 2 - PROGRESS: at 82.67% examples, 369403 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:54:26,820 : INFO : EPOCH 2 - PROGRESS: at 85.34% examples, 368459 words/s, in_qsize 8, out_qsize 0
2018-03-03 18:54:27,837 : INFO : EPOCH 2 - PROGRESS: at 88.52% examples, 369526 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:54:28,914 : INFO : EPOCH 2 - PROGRESS: at 91.26% examples, 368028 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:54:29,917 : INFO : EPOCH 2 - PROGRESS: at 94.43% examples, 368998 words/s, in_qsize 8, out_qsize 0
2018-03-03 18:54:30,924 : INFO : EPOCH 2 - PROGRESS: at 97.67% examples, 370284 words/s, in_qsize 8, out_qsize 0
2018-03-03 18:54:31,621 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-

2018-03-03 18:55:30,306 : INFO : EPOCH 4 - PROGRESS: at 67.69% examples, 351674 words/s, in_qsize 8, out_qsize 0
2018-03-03 18:55:31,322 : INFO : EPOCH 4 - PROGRESS: at 71.49% examples, 356754 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:55:32,338 : INFO : EPOCH 4 - PROGRESS: at 74.46% examples, 357379 words/s, in_qsize 8, out_qsize 1
2018-03-03 18:55:33,348 : INFO : EPOCH 4 - PROGRESS: at 78.16% examples, 361397 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:55:34,357 : INFO : EPOCH 4 - PROGRESS: at 82.04% examples, 365907 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:55:35,358 : INFO : EPOCH 4 - PROGRESS: at 85.96% examples, 370464 words/s, in_qsize 7, out_qsize 0
2018-03-03 18:55:36,372 : INFO : EPOCH 4 - PROGRESS: at 89.63% examples, 373609 words/s, in_qsize 7, out_qsize 1
2018-03-03 18:55:37,382 : INFO : EPOCH 4 - PROGRESS: at 93.51% examples, 377269 words/s, in_qsize 6, out_qsize 1
2018-03-03 18:55:38,387 : INFO : EPOCH 4 - PROGRESS: at 97.23% examples, 380124 words/s, in_qsiz

In [15]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2018-03-03 18:58:11,641 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-03-03 18:58:12,446 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-03-03 18:58:12,448 : INFO : setting ignored attribute vectors_norm to None
2018-03-03 18:58:12,449 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-03-03 18:58:12,450 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-03-03 18:58:12,452 : INFO : setting ignored attribute cum_table to None
2018-03-03 18:58:12,454 : INFO : loaded 300features_40minwords_10context


In [20]:
type(model.wv.syn0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [28]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print ("Review %d of %d" , (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [29]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review %d of %d (0, 25000)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review %d of %d (1000, 25000)
Review %d of %d (2000, 25000)
Review %d of %d (3000, 25000)
Review %d of %d (4000, 25000)
Review %d of %d (5000, 25000)
Review %d of %d (6000, 25000)
Review %d of %d (7000, 25000)
Review %d of %d (8000, 25000)
Review %d of %d (9000, 25000)
Review %d of %d (10000, 25000)
Review %d of %d (11000, 25000)
Review %d of %d (12000, 25000)
Review %d of %d (13000, 25000)
Review %d of %d (14000, 25000)
Review %d of %d (15000, 25000)
Review %d of %d (16000, 25000)
Review %d of %d (17000, 25000)
Review %d of %d (18000, 25000)
Review %d of %d (19000, 25000)
Review %d of %d (20000, 25000)
Review %d of %d (21000, 25000)
Review %d of %d (22000, 25000)
Review %d of %d (23000, 25000)
Review %d of %d (24000, 25000)
Creating average feature vecs for test reviews
Review %d of %d (0, 25000)
Review %d of %d (1000, 25000)
Review %d of %d (2000, 25000)
Review %d of %d (3000, 25000)
Review %d of %d (4000, 25000)
Review %d of %d (5000, 25000)
Review %d of %d (6000, 25000)
Review %d o

In [30]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
